# Exploratory Data Anaysis Order Items Dataset

In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
import hashlib
import os
from datetime import datetime
import re
os.environ["AWS_PROFILE"] = "de_jay_east"


In [2]:

!aws s3 ls s3://gp-elt-657082399901-dev/

                           PRE drivers/
                           PRE final/
                           PRE mapping/
                           PRE metrics/
                           PRE processed/
                           PRE quality/
                           PRE scripts/
                           PRE transform/
2025-08-01 22:08:03          0 metrics_$folder$
2025-08-01 22:06:25          0 quality_$folder$
2025-08-01 22:04:03          0 transform_$folder$


In [105]:
# !aws s3 rm s3://gp-elt-657082399901-dev/metrics/sales_trends/ --recursive

In [107]:
# !aws s3 ls s3://gp-elt-657082399901-dev/metrics/sales_trends/daily/
#

In [ ]:
# !aws s3 ls s3://gp-elt-657082399901-dev/quality/ --recursive --human-readable --summarize

In [21]:
# !aws s3 ls s3://gp-elt-657082399901-dev/final/ --recursive --human-readable --summarize

In [22]:
# !aws s3 rm s3://gp-elt-657082399901-dev/quality/ --recursive


In [23]:
# !aws s3 rm s3://gp-elt-657082399901-dev/final/ --recursive --exclude "*" --include "*.parquet"

In [ ]:
# Order Items info
order_items = pd.read_csv('order_items.csv')
# make dataframe columns lower caps
order_items.columns = order_items.columns.str.lower()



In [60]:
# Order Items info
order_item_options = pd.read_csv('order_item_options.csv')
# make dataframe columns lower caps
order_item_options.columns = order_item_options.columns.str.lower()



In [40]:
# Order Items distinct restaurants
print(f"Distinct Restaurants: {order_items['restaurant_id'].nunique()}")


Distinct Restaurants: 28


In [41]:
# distinct item categories
print("Distinct Item Categories:")
distinct_categories = order_items['item_category'].unique()
distinct_categories = pd.Series(distinct_categories).sort_values().reset_index(drop=True)
distinct_categories = distinct_categories[distinct_categories.notnull()]
distinct_categories = distinct_categories[distinct_categories != '']
distinct_categories = distinct_categories[distinct_categories != 'nan']
distinct_categories = distinct_categories[distinct_categories != 'None']
distinct_categories = distinct_categories[distinct_categories != 'none']
distinct_categories = distinct_categories[distinct_categories != 'nan ']
distinct_categories = distinct_categories[distinct_categories != 'none ']
distinct_categories = distinct_categories[distinct_categories != 'nan']
distinct_categories = distinct_categories[distinct_categories != 'none']        
print(distinct_categories)


Distinct Item Categories:
0                                            BBQ Plates
1     BBQ Plateshttps://order.pxsweb.com/admin/resta...
2                                    BBQ Sides & Extras
3                                     Bottled Beverages
4                                                 Bowls
5                                                Bowls0
6                                             Breakfast
7                                  Burgers & Sandwiches
8                                     Candy & Chocolate
9                                              Catering
10                                                Chips
11                                            Cold Brew
12                                               Drinks
13    Drip Chttps://www.opendining.net/admin/restaur...
14                                          Drip Coffee
15                                        Energy Boosts
16                                             Espresso
17                    

In [42]:
# distinct item restaurants
print("Distinct Item Restaurants:")
distinct_restaurants = order_items['restaurant_id'].unique()
distinct_restaurants = order_items['restaurant_id'].unique()
distinct_restaurants = pd.Series(distinct_restaurants).sort_values().reset_index(drop=True)
distinct_restaurants = distinct_restaurants[distinct_restaurants.notnull()]
distinct_restaurants = distinct_restaurants[distinct_restaurants != '']
distinct_restaurants = distinct_restaurants[distinct_restaurants != 'nan']
distinct_restaurants = distinct_restaurants[distinct_restaurants != 'None']
distinct_restaurants = distinct_restaurants[distinct_restaurants != 'none']
distinct_restaurants = distinct_restaurants[distinct_restaurants != 'nan ']
distinct_restaurants = distinct_restaurants[distinct_restaurants != 'none ']
distinct_restaurants = distinct_restaurants[distinct_restaurants != 'nan']
distinct_restaurants = distinct_restaurants[distinct_restaurants != 'none'] 
print(distinct_restaurants)

Distinct Item Restaurants:
0     5e7e35ec902ad5ac017b242a
1     5e7e35ed902ad5ac017b242b
2     5e7e35ef902ad5ac017b242c
3     5e7e35f0902ad5ac017b242d
4     5e7e36f4505ee9fc797b23ed
5     5ed54933902ad53b4128658b
6     5ed55504505ee9356dda2ff6
7     5edff2fe4f5ee969161e8404
8     5edff313902ad5580e9c3f30
9     5f6a6c1537ab46bd38e9df75
10    5fece84aadb0d51509c36f22
11    5fff32f8adb0d510048f8bfa
12    6050e76361e498ca740bba6f
13    6054db3295b70198148b456d
14    6054db3395b70198148b456e
15    6054db3495b70198148b456f
16    60b6544461e4985712e053c6
17    60c9339862e49873042c2f64
18    61bccd6a7b92a75dc16baf4e
19    622107b40ac81503e0369ca6
20    622289bc6863d23d066d56ff
21    6234ae308b054a7c4c685ef5
22    62603c5b3fbb945d37551125
23    62754fb697ef8a71910f97bd
24    62f2ce9824813746ce6f5140
25    6351b7d1a036663f05239b60
26    63bc98a7519adc105105a990
27    64920f827e0a9111a402ef4a
dtype: object


In [43]:
# Detect unusual category labels with URLs, special characters, or abnormal length
def is_weird_category(cat):
    if not isinstance(cat, str):
        return False
    return (
        'http' in cat.lower() or
        '`' in cat or
        len(cat.strip()) > 30
    )

# Apply the filter
weird_labels = [cat for cat in order_items['item_category'].unique() if is_weird_category(cat)]

# Display the results
print("Suspicious or malformed item_category labels:")
for label in weird_labels:
    print(f"- {label}")
    

Suspicious or malformed item_category labels:
- BBQ Plateshttps://order.pxsweb.com/admin/restaurants/622289bc6863d23d066d56ff/menuitems#
- Drip Chttps://www.opendining.net/admin/restaurants/62f2ce9824813746ce6f5140/menuitems#offee
- Sandwiches`1
- Kid'shttps://www.opendining.net/admin/restaurants/6054db3295b70198148b456d/menuitems#


In [44]:
def clean_category_label(label):
    if not isinstance(label, str):
        return label
    # Remove URLs
    label = re.sub(r'http\S+', '', label)
    # Remove backticks and extra whitespace
    label = label.replace('`', '').strip()
    # Optionally truncate very long labels
    return label[:30]  # Keep first 30 characters if still too long

# Apply cleaning function to item_category
order_items['item_category'] = order_items['item_category'].apply(clean_category_label)

# Optional: View top unique cleaned categories
print("Unique cleaned categories:")
#print(order_items['item_category'].value_counts())


Unique cleaned categories:


In [ ]:
# order_items[order_items['item_category'].str.contains('breakfast', case=False, na=False)] 
# List all unique item_name values in breakfast category
unique_breakfast_items = (
    order_items[order_items['item_category'].str.contains('breakfast', case=False, na=False)]
    ['item_name']
    .dropna()
    .unique()
)

for item in unique_breakfast_items.tolist():
    print(item.lower().strip())


bacon egg & cheese*
breakfast burrito*
southwest sausage breakfast sandwich*
smoked ham & hot honey egg & cheese*
maple sausage egg & cheese*
cracked oats oatmeal
chilaquiles*
breakfast nacho*
avocado toast
pork roll*
bacon egg & cheese
loaded bagel
fresh classic
toasted bagel with cream cheese
vegan breakfast burrito
baja burrito vegan
fresh classic* | veg
southwest scramble* |  veg, gf
baja burrito* | veg
mindful moroccan
fresh classic
baja burrito vegan |  v, veg
brooklyn bliss | veg
shakshuka*
egg & cheese bacon
green smash
maine grain oats
maine grain oats | v, veg, gf
shakshuka* | veg
baja burrito
green smash | v, veg
test
southwest scramble vegan* | v, veg, gf
shakshuka
totes oats
southwest scramble
egg & cheese ham
southwest scramble* veg, gf
egg & cheese
buddha bowl
cracked oatmeal
brooklyn bliss
egg & cheese sausage
breakfast burrito
baja burrito
shakshuka
fresh classic* | veg
southwest scramble vegan
maine grain oats
the shakshuka
avocado toast
chilaquiles


In [69]:
category_list = (
    order_items['item_category'].dropna().unique())
# print(category_list)

# unique_breakfast_items = (
#     order_items[order_items['item_category'].str.contains('breakfast', case=False, na=False)]
#     ['item_name_cleaned']
#     .dropna()
#     .unique()
# )

for category in category_list.tolist():
    print(category.lower().strip())


breakfast
salads
sandwiches
bowls
smoothies
chips
specialty coffee beverages
drip coffee
bottled beverages
kid's
subs
drinks
bbq plates
bbq sides & extras
most popular
soda, juices & kombucha
candy & chocolate
energy boosts
milk
drip c
espresso
hot espresso
kids
meal prep kits
catering
vegetarian options
bowls0
gluten free options
iced espresso
cold brew
hot bowls
sqalads
snacks
pizza
burgers & sandwiches
sandwiches1
specialty coffee
pre-made sandwiches
vegan options
iced coffee
test items
hot coffee
tobacco


In [46]:
import re

def clean_item_name_and_extract_patterns(item_name):
    """
    Remove all '*' characters from the item_name and extract any *pattern* if fully enclosed.
    
    Args:
        item_name (str): The item name with or without *.

    Returns:
        tuple: (cleaned_item_name, patterns_matched_within_asterisks)
    """
    if not isinstance(item_name, str):
        return item_name, []

    # Extract patterns like *something*
    patterns = re.findall(r'\*([^*]+)\*', item_name)

    # Remove all * characters regardless of position
    cleaned_name = item_name.replace('*', '').strip().lower()

    return cleaned_name, patterns

# Get unique breakfast item names
unique_breakfast_items = (
    order_items[order_items['item_category'].str.contains('breakfast', case=False, na=False)]
    ['item_name']
    .dropna()
    .unique()
)

# Apply cleaning and pattern extraction
for item in unique_breakfast_items.tolist():
    cleaned_item, patterns = clean_item_name_and_extract_patterns(item)
    print(cleaned_item)
    if patterns:
        print(f"  ↳ Removed pattern(s): {patterns}")


bacon egg & cheese
breakfast burrito
southwest sausage breakfast sandwich
smoked ham & hot honey egg & cheese
maple sausage egg & cheese
cracked oats oatmeal
chilaquiles
breakfast nacho
avocado toast
pork roll
bacon egg & cheese
loaded bagel
fresh classic
toasted bagel with cream cheese
vegan breakfast burrito
baja burrito vegan
fresh classic | veg
southwest scramble |  veg, gf
baja burrito | veg
mindful moroccan
fresh classic
baja burrito vegan |  v, veg
brooklyn bliss | veg
shakshuka
egg & cheese bacon
green smash
maine grain oats
maine grain oats | v, veg, gf
shakshuka | veg
baja burrito
green smash | v, veg
test
southwest scramble vegan | v, veg, gf
shakshuka
totes oats
southwest scramble
egg & cheese ham
southwest scramble veg, gf
egg & cheese
buddha bowl
cracked oatmeal
brooklyn bliss
egg & cheese sausage
breakfast burrito
baja burrito
shakshuka
fresh classic | veg
southwest scramble vegan
maine grain oats
the shakshuka
avocado toast
chilaquiles


In [47]:
import re
import pandas as pd

def clean_item_name_and_extract_patterns(item_name):
    """
    Remove all '*' characters from the item_name and extract any *pattern* if fully enclosed.
    
    Args:
        item_name (str): The item name with or without *.

    Returns:
        tuple: (cleaned_item_name, patterns_matched_within_asterisks)
    """
    if not isinstance(item_name, str):
        return item_name, []

    # Extract patterns like *something*
    patterns = re.findall(r'\*([^*]+)\*', item_name)

    # Remove all * characters regardless of position
    cleaned_name = item_name.replace('*', '').strip().lower()

    return cleaned_name, patterns


# --- Apply to the full order_items DataFrame ---

# Create two new columns: one for cleaned names, one for extracted patterns
order_items[['item_name_cleaned', 'extracted_patterns']] = (
    order_items['item_name']
    .apply(lambda x: pd.Series(clean_item_name_and_extract_patterns(x)))
)

# If you want only the breakfast rows:
# breakfast_df = order_items[order_items['item_category'].str.contains('breakfast', case=False, na=False)]

# Show a few rows
print(order_items[['item_name', 'item_name_cleaned', 'extracted_patterns']].head())


             item_name   item_name_cleaned extracted_patterns
0  Bacon Egg & Cheese*  bacon egg & cheese                 []
1   Breakfast Burrito*   breakfast burrito                 []
2  Bacon Egg & Cheese*  bacon egg & cheese                 []
3          Greek Salad         greek salad                 []
4   Breakfast Burrito*   breakfast burrito                 []


In [48]:
order_items.head()

,app_name,restaurant_id,creation_time_utc,order_id,user_id,printed_card_number,is_loyalty,currency,lineitem_id,item_category,item_name,item_price,item_quantity,item_name_cleaned,extracted_patterns
0,Alltown Fresh,63bc98a7519adc105105a990,2023-03-08T11:03:32.223Z,64086b8463905bf310002079,63ee144950286a8367041911,NaN,False,USD,64086b84d5fbd8718a0717e6,Breakfast,Bacon Egg & Cheese*,5.99,1,bacon egg & cheese,[]
1,Alltown Fresh,5f6a6c1537ab46bd38e9df75,2023-05-13T12:45:00.475Z,645f864c1c4b0312f10cfa75,NaN,NaN,False,USD,645f869568ab8e534e099ea7,Breakfast,Breakfast Burrito*,7.99,1,breakfast burrito,[]
2,Alltown Fresh,5fece84aadb0d51509c36f22,2023-02-03T07:15:28.519Z,63dcb490665d80c6be09a618,5ece77fe902ad501337b23fd,NaN,False,USD,63dcb49b665d80c6be09a657,Breakfast,Bacon Egg & Cheese*,5.99,1,bacon egg & cheese,[]
3,Alltown Fresh,5f6a6c1537ab46bd38e9df75,2023-08-17T16:18:02.231Z,64de483a7028adc2520d8cf0,NaN,NaN,False,USD,64de484eb7b035d89d004141,Salads,Greek Salad,11.99,1,greek salad,[]
4,Alltown Fresh,622107b40ac81503e0369ca6,2023-08-20T16:02:12.518Z,64e23904baef31a741016be9,NaN,NaN,False,USD,64e2395465826c9b650c1c37,Breakfast,Breakfast Burrito*,5.99,1,breakfast burrito,[]


In [49]:
# order_items[order_items['item_category'].str.contains('breakfast', case=False, na=False)] 
# List all unique item_name values in breakfast category
unique_breakfast_items = (
    order_items[order_items['item_category'].str.contains('breakfast', case=False, na=False)]
    ['item_name_cleaned']
    .dropna()
    .unique()
)

for item in unique_breakfast_items.tolist():
    print(item.lower().strip())

bacon egg & cheese
breakfast burrito
southwest sausage breakfast sandwich
smoked ham & hot honey egg & cheese
maple sausage egg & cheese
cracked oats oatmeal
chilaquiles
breakfast nacho
avocado toast
pork roll
loaded bagel
fresh classic
toasted bagel with cream cheese
vegan breakfast burrito
baja burrito vegan
fresh classic | veg
southwest scramble |  veg, gf
baja burrito | veg
mindful moroccan
baja burrito vegan |  v, veg
brooklyn bliss | veg
shakshuka
egg & cheese bacon
green smash
maine grain oats
maine grain oats | v, veg, gf
shakshuka | veg
baja burrito
green smash | v, veg
test
southwest scramble vegan | v, veg, gf
totes oats
southwest scramble
egg & cheese ham
southwest scramble veg, gf
egg & cheese
buddha bowl
cracked oatmeal
brooklyn bliss
egg & cheese sausage
southwest scramble vegan
the shakshuka


In [57]:
filtered_prices = (
    order_items[
        (order_items['item_category'].str.contains('breakfast', case=False, na=False)) &
        (order_items['item_name_cleaned'].str.contains('avocado toast', case=False, na=False)) &
        (order_items['item_quantity'] == 1)
    ][['item_name_cleaned', 'item_price', 'item_quantity', 'lineitem_id']]
    .dropna()
    .drop_duplicates()
    .sort_values('item_price')
)

print(filtered_prices)


       item_name_cleaned  item_price  item_quantity               lineitem_id
15800      avocado toast        0.89              1  60741e7162e498e123257e3b
48127      avocado toast        4.89              1  619e350662e498aa62347ca7
9428       avocado toast        4.89              1  6116626762e498b758103467
195192     avocado toast        4.89              1  61827db261e498c8362c4305
92197      avocado toast        4.89              1  606af48661e4989207d732b3
...                  ...         ...            ...                       ...
201479     avocado toast        9.89              1  62cbf978f110d07dac74c01d
186457     avocado toast        9.99              1  654795e1886123b73a0463df
154932     avocado toast        9.99              1  6404b25a252819dd120b5995
153877     avocado toast       10.99              1  64ad36890d283c214005c6eb
74880      avocado toast       13.89              1  6243e1709ebfa372946f6411

[3394 rows x 4 columns]


In [65]:
order_item_options[order_item_options['lineitem_id'].str.contains('64ad36890d283c214005c6eb', case=False, na=False)]

,order_id,lineitem_id,option_group_name,option_name,option_price,option_quantity
96624,64ad368720b0dbc87405453a,64ad36890d283c214005c6eb,Avocado Toast Options,Add Egg Salad,1.0,1
96625,64ad368720b0dbc87405453a,64ad36890d283c214005c6eb,Avocado Toast Options,Add Smoked Salmon,2.0,1
96626,64ad368720b0dbc87405453a,64ad36890d283c214005c6eb,Avocado Toast Options,Add Smokehouse Bacon,2.0,1


In [115]:

# lets group by restaurant and item category
order_items_grouped = order_items.groupby(['restaurant_id', 'item_category']).agg({
    'item_quantity': 'sum',
    'item_price': 'mean',
    'item_name': 'count'
}).reset_index()
order_items_grouped.tail(20)

,restaurant_id,item_category,item_quantity,item_price,item_name
319,63bc98a7519adc105105a990,Bottled Beverages,187,2.199605,177
320,63bc98a7519adc105105a990,Bowls,864,12.368161,843
321,63bc98a7519adc105105a990,Breakfast,3509,6.922763,3225
322,63bc98a7519adc105105a990,Chips,203,2.184932,148
323,63bc98a7519adc105105a990,Drip Coffee,457,3.140093,431
324,63bc98a7519adc105105a990,Kid's,111,5.208476,105
325,63bc98a7519adc105105a990,Salads,594,11.528700,577
326,63bc98a7519adc105105a990,Sandwiches,3426,11.464657,3247
327,63bc98a7519adc105105a990,Smoothies,1363,8.872192,1291
328,63bc98a7519adc105105a990,"Soda, Juices & Kombucha",5,4.890000,5


In [116]:
order_items[order_items['item_category'].str.contains('drip coffee', case=False, na=False)]

,app_name,restaurant_id,creation_time_utc,order_id,user_id,printed_card_number,is_loyalty,currency,lineitem_id,item_category,item_name,item_price,item_quantity
34,Alltown Fresh,6351b7d1a036663f05239b60,2023-05-31T01:03:20.653Z,64769cd824737fbab109eb15,NaN,NaN,False,USD,64769cfb4ddec677180339cf,Drip Coffee,ALLTOWN FRESH HOT COFFEE,2.79,1
38,Alltown Fresh,63bc98a7519adc105105a990,2023-07-26T10:43:34.982Z,64c0f8d694cb4ae3fc01f314,63e709d95d6b4353060c70d9,6.780308e+11,True,USD,64c0f91b7f6150834a0a4157,Drip Coffee,ALLTOWN FRESH HOT COFFEE,2.49,1
52,Alltown Fresh,6054db3395b70198148b456e,2023-12-20T13:51:52.08Z,6582f178dfed112cd70e299e,62221b6f4e3c275d216a774e,6.780087e+11,True,USD,6582f2c442b1fe2cac07caaa,Drip Coffee,ALLTOWN FRESH HOT COFFEE,3.24,1
88,Alltown Fresh,5e7e35ec902ad5ac017b242a,2023-03-03T11:58:07.006Z,6401e0cfa7781c95100a05f7,61432ef362e4988f7f72b6e4,NaN,False,USD,6401e0e7b5022a67f704ca69,Drip Coffee,ALLTOWN FRESH ICED COFFEE,3.39,1
105,Alltown Fresh,5e7e35ec902ad5ac017b242a,2023-09-07T10:46:18.232Z,64f9a9fa68386e7550086fef,604a361b4f5ee9e1523d65b5,6.780007e+11,True,USD,64f9aa4bd77025f4ff0b3bbd,Drip Coffee,ALLTOWN FRESH HOT COFFEE,2.49,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
203371,Alltown Fresh,622289bc6863d23d066d56ff,2022-06-07T11:24:12.275Z,629f355c86f5a703d2654ee4,628411d658cf8c54bd608a59,NaN,False,USD,629f35a2665a1a3c6a30f61f,Drip Coffee,ALLTOWN FRESH ICED COFFEE,2.89,1
203435,Alltown Fresh,5f6a6c1537ab46bd38e9df75,2022-07-20T09:55:57.636Z,62d7d12dafdf17444b0902ab,61562930ffd1251d14e7a8c0,NaN,False,USD,62d7d17ec5567f276f03843e,Drip Coffee,ALLTOWN FRESH ICED COFFEE,3.94,1
203454,Alltown Fresh,5e7e35ed902ad5ac017b242b,2023-06-10T14:29:18.231Z,648488be3c189169b409315a,609d681462e498b356e72a6d,6.780027e+11,True,USD,648488d53bd67e52f20bf234,Drip Coffee,ALLTOWN FRESH HOT COFFEE,2.79,1
203474,Alltown Fresh,5fece84aadb0d51509c36f22,2024-01-03T12:34:38.912Z,6595545e9ca1d047b00f69f3,6421c7707ecddfa0a90519c5,6.780346e+11,True,USD,6595546d8417b4bbba090974,Drip Coffee,ALLTOWN FRESH ICED COFFEE,2.99,1


# Troubleshooting Steps: Quantity Outliers

In [119]:
import pandas as pd
import numpy as np

def bin_item_quantities(df, column="item_quantity"):
    """
    Adds a 'quantity_bin' column to the DataFrame based on defined quantity bins.
    
    Parameters:
        df (pd.DataFrame): Input DataFrame
        column (str): Column to bin (default: 'item_quantity')
        
    Returns:
        pd.DataFrame: Updated DataFrame with 'quantity_bin' column
    """
    bins = [0, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, np.inf]
    labels = [f">{bins[i]}" for i in range(len(bins)-1)]
    
    df["quantity_bin"] = pd.cut(df[column], bins=bins, labels=labels, right=False)
    return df


In [120]:
import plotly.express as px

fig1 = px.histogram(
    order_short, 
    x="item_quantity", 
    nbins=30, 
    marginal="violin",  # or 'rug', 'box', 'violin' for additional info
    opacity=0.7,
    title="Distribution of Item Quantities with KDE",
    labels={"item_quantity": "Item Quantity"}
)
fig1.update_layout(bargap=0.1)
fig1.show()


In [121]:
fig2 = px.box(
    order_short, 
    x="item_quantity", 
    title="Boxplot of Item Quantities",
    labels={"item_quantity": "Item Quantity"},
    points="all"  # show outliers as individual points
)
fig2.show()


In [122]:
fig2 = px.box(
    order_short, 
    x="item_quantity", 
    y="item_category",
    title="Boxplot of Item Quantities",
    labels={"item_quantity": "Item Quantity"},
    points="all"  # show outliers as individual points
)
fig2.show()

In [123]:
order_items[order_items['item_quantity'] > 25]

,app_name,restaurant_id,creation_time_utc,order_id,user_id,printed_card_number,is_loyalty,currency,lineitem_id,item_category,item_name,item_price,item_quantity
5851,Alltown Fresh,5e7e35f0902ad5ac017b242d,2020-08-04T14:52:13.64Z,5f29761d37ab466741a492c7,5eb44c8c902ad51d3c7b23d6,NaN,False,USD,5f29761d37ab466741a492c9,Hot Espresso,Mocha - 12oz,70.20,27
11399,Alltown Fresh,5edff313902ad5580e9c3f30,2020-11-03T03:45:43.156Z,5fa0d26737ab46f061db30e9,5fa0216b4f5ee9b5352822e5,NaN,False,USD,5fa0d267adb0d5675a4638b9,Sandwiches,Turnt Up Tuna,959.00,106
15769,Alltown Fresh,5e7e35f0902ad5ac017b242d,2020-06-02T08:46:00.719Z,5ed611c8505ee9cc68872ead,5ecf9eda505ee9682b445912,NaN,False,USD,5ed611ca505ee90b68872ebb,Hot Espresso,Hot Chocolate - 12oz,245.00,50
30262,Alltown Fresh,5edff313902ad5580e9c3f30,2021-03-03T18:57:29.844Z,603fdc19505ee91d1eb8cce2,603fdbf837ab46cf2eec0150,NaN,False,USD,603fdcd5adb0d5a12db8ccdf,Bottled Beverages,COCA COLA 20OZ,142.35,65
40323,Alltown Fresh,5e7e35f0902ad5ac017b242d,2020-10-08T13:23:30.454Z,5f7f12d2505ee9af5f7b23f5,5f7ef1a64f5ee90e168223cb,NaN,False,USD,5f7f12d34f5ee99e4e7b23dc,Catering,PLATTER BAGELS,75.00,30
50708,Alltown Fresh,5e7e35ed902ad5ac017b242b,2023-05-27T21:30:21.372Z,6472766d93aa377a70078c9a,NaN,NaN,False,USD,647276fb3ac5d568f704381f,Salads,Southwest Chop Salad,269.73,27
82855,Alltown Fresh,5e7e35f0902ad5ac017b242d,2020-12-29T16:39:38.142Z,5feb5bca4f5ee968184783b9,5f3c1860505ee9de2a7b23f0,NaN,False,USD,5feb5bcaadb0d54f6e3ed2fb,Bowls,Chili Chicken | GF,364.00,28
111542,Alltown Fresh,5e7e35f0902ad5ac017b242d,2020-06-03T12:27:22.555Z,5ed7972a4f5ee90547011c03,5ecf9eda505ee9682b445912,NaN,False,USD,5ed799d6505ee98a68cde72d,Espresso,Espresso - Double,1125.00,500
111543,Alltown Fresh,5e7e35f0902ad5ac017b242d,2020-06-03T12:27:22.555Z,5ed7972a4f5ee90547011c03,5ecf9eda505ee9682b445912,NaN,False,USD,5ed79a9d505ee91a69cde731,Smoothies,Meet Your Matcha - 12oz,3500.00,500
118704,Alltown Fresh,5e7e35f0902ad5ac017b242d,2020-08-19T12:31:35.028Z,5f3d1ba7505ee9374b7b23cc,5ee7285f505ee9f7490cf93d,NaN,False,USD,5f3d1ba84f5ee91b687b23ec,Bowls,Korean Kimchi,5000.00,500


In [124]:
order_items[
    (order_items['item_category'] == 'Specialty Coffee') & 
    (order_items['item_name'].str.contains('mocha', case=False, na=False))
].sort_values(by="item_price", ascending=False)


,app_name,restaurant_id,creation_time_utc,order_id,user_id,printed_card_number,is_loyalty,currency,lineitem_id,item_category,item_name,item_price,item_quantity


In [125]:
import pandas as pd
import numpy as np

def bin_item_quantities(df, column="item_price"):
    """
    Adds a 'quantity_bin' column to the DataFrame based on defined quantity bins.
    
    Parameters:
        df (pd.DataFrame): Input DataFrame
        column (str): Column to bin (default: 'item_quantity')
        
    Returns:
        pd.DataFrame: Updated DataFrame with 'quantity_bin' column
    """
    bins = [2,4,6,8,10,12,14,16,18,20, np.inf]
    labels = [f">{bins[i]}" for i in range(len(bins)-1)]
    
    df["price_bin"] = pd.cut(df[column], bins=bins, labels=labels, right=False)
    return df

In [126]:
import plotly.express as px

fig1 = px.histogram(
    order_short, 
    x="item_price", 
    nbins=30, 
    marginal="violin",  # or 'rug', 'box', 'violin' for additional info
    opacity=0.7,
    title="Distribution of Item Prices with KDE",
    labels={"item_price": "Item Price"}
)
fig1.update_layout(bargap=0.1)
fig1.show()


In [127]:
order_items[order_items['item_price'] > 90]

,app_name,restaurant_id,creation_time_utc,order_id,user_id,printed_card_number,is_loyalty,currency,lineitem_id,item_category,item_name,item_price,item_quantity
2326,Alltown Fresh,5e7e35ec902ad5ac017b242a,2020-12-04T09:58:02.187Z,5fca082a505ee96343ab2662,5fca07cf505ee96b44ab264f,NaN,False,USD,5fca082b4f5ee9a852d0ec00,Bowls,"Korean Kimchi | V, VEG, GF",100.0,10
7264,Alltown Fresh,5e7e35ec902ad5ac017b242a,2020-12-23T13:04:55.944Z,5fe3407737ab46070d40a6ab,5ecf9eda505ee9682b445912,NaN,False,USD,5fe34079505ee9296d4e7e20,Meal Prep Kits,Fresh Fiesta Taco Night,240.0,4
8830,Alltown Fresh,5e7e35ed902ad5ac017b242b,2020-12-07T07:29:43.701Z,5fcdd9e74f5ee9885c07c4bc,5fcc79d04f5ee9f812be49c2,NaN,False,USD,5fcddaaa505ee96d7a530bae,Meal Prep Kits,Homestyle Roast Chicken Dinner,240.0,4
10467,Alltown Fresh,5edff313902ad5580e9c3f30,2021-02-19T20:44:36.539Z,6030233437ab46737a50f63e,5f3c1860505ee9de2a7b23f0,NaN,False,USD,603023343f8e26b065c2a000,Sandwiches,Smoked B.L.A.T,140.0,10
10571,Alltown Fresh,5e7e35ec902ad5ac017b242a,2020-12-04T09:58:02.455Z,5fca082a505ee9bf42ab266b,5fca07cf505ee96b44ab264f,NaN,False,USD,5fca082c505ee9ad43ab2652,Bowls,"Korean Kimchi | V, VEG, GF",106.0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
191283,Alltown Fresh,5edff313902ad5580e9c3f30,2021-03-03T18:53:47.396Z,603fdb3badb0d50325b8cd61,5f3c1860505ee9de2a7b23f0,NaN,False,USD,603fdb3b505ee9741db8ccd2,Meal Prep Kits,Comfortable And Cozy,540.0,6
193563,Alltown Fresh,5edff313902ad5580e9c3f30,2020-09-14T16:07:04.959Z,5f5f952837ab463a7e7b23e0,5f1b00e5535ee93e0cb768e7,NaN,False,USD,5f5f95294f5ee99d687b23c9,Catering,SALAD KILLER COBB Large,315.0,7
197418,Alltown Fresh,5edff313902ad5580e9c3f30,2020-12-11T15:50:32.315Z,5fd39548535ee9a1230f35f8,5fcf926badb0d5cf35a1f8ca,NaN,False,USD,5fd39597adb0d5fc51c4c0e7,Meal Prep Kits,Steak House Dinner,150.0,1
198388,Alltown Fresh,5e7e35f0902ad5ac017b242d,2020-06-17T14:25:46.299Z,5eea27ea902ad54633544d34,NaN,NaN,False,USD,5eea27ec902ad53134544d2e,Smoothies,Power - 12oz,170.0,10


In [129]:
import pandas as pd
import numpy as np

def bin_item_quantities(df, column="item_price"):
    """
    Adds a 'quantity_bin' column to the DataFrame based on defined quantity bins.
    
    Parameters:
        df (pd.DataFrame): Input DataFrame
        column (str): Column to bin (default: 'item_quantity')
        
    Returns:
        pd.DataFrame: Updated DataFrame with 'quantity_bin' column
    """
    bins = [2,4,6,8,10,12,14,16,18,20, np.inf]
    labels = [f">{bins[i]}" for i in range(len(bins)-1)]
    
    df["price_bin"] = pd.cut(df[column], bins=bins, labels=labels, right=False)
    return df

In [130]:
import plotly.express as px

fig1 = px.histogram(
    order_short, 
    x="item_price",  
    y="item_category",
    nbins=30, 
    marginal="violin",  # or 'rug', 'box', 'violin' for additional info
    opacity=0.7,
    title="Distribution of Item Prices with KDE",
    labels={"item_price": "Item Price"}
)
fig1.update_layout(bargap=0.1)
fig1.show()


In [131]:
order_short[order_short["item_category"]=='Sandwiches']

,item_category,lineitem_id,item_name,item_price,item_quantity


In [132]:
import plotly.express as px

fig1 = px.histogram(
    order_short, 
    x="item_price", 
    y="item_category",
    nbins=30, 
    marginal="violin",  # or 'rug', 'box', 'violin' for additional info
    opacity=0.7,
    title="Distribution of Item Prices with KDE",
    labels={"item_price": "Item Price"}
)
fig1.update_layout(bargap=0.1)
fig1.show()


In [134]:
order_items[ (order_items['item_category'] =='Catering') &(order_items['item_name'] =='SIDE FRUIT SM')].head(7)

,app_name,restaurant_id,creation_time_utc,order_id,user_id,printed_card_number,is_loyalty,currency,lineitem_id,item_category,item_name,item_price,item_quantity
53166,Alltown Fresh,5f6a6c1537ab46bd38e9df75,2020-11-03T10:14:53.456Z,5fa12d9d4f5ee90948197efc,5f9fb66a4f5ee98e0fe32c38,NaN,False,USD,5fa12e11505ee935705ed404,Catering,SIDE FRUIT SM,400.0,10
57865,Alltown Fresh,5e7e35ec902ad5ac017b242a,2020-11-02T14:55:14.363Z,5fa01dd24f5ee9be2f2822e4,5fa01c53505ee92468cbdf63,NaN,False,USD,5fa01eb437ab46a33cc6f775,Catering,SIDE FRUIT SM,80.0,2
105195,Alltown Fresh,5edff313902ad5580e9c3f30,2020-10-23T12:52:22.08Z,5f92d2064f5ee9e00ecbd029,5f1b00e5535ee93e0cb768e7,NaN,False,USD,5f92d218505ee9914df674c5,Catering,SIDE FRUIT SM,28.0,1
139558,Alltown Fresh,5edff313902ad5580e9c3f30,2020-11-02T20:45:43.248Z,5fa06ff7505ee92a23de4a7b,5fa01c53505ee92468cbdf63,NaN,False,USD,5fa07050adb0d5f4784638d9,Catering,SIDE FRUIT SM,120.0,3
177418,Alltown Fresh,5edff313902ad5580e9c3f30,2020-11-02T20:53:56.953Z,5fa071e4505ee90226de4a83,5fa01c53505ee92468cbdf63,NaN,False,USD,5fa071e5505ee90926de4a73,Catering,SIDE FRUIT SM,400.0,10


In [135]:
# Step 1: Filter Catering items
catering_items = order_items[order_items['item_category'].str.lower() == 'catering']

# Step 2–3: Group by item_name and collect distinct price and quantity
item_consistency = (
    catering_items
    .groupby('item_name')
    .agg({
        'item_price': lambda x: sorted(set(x)),
        'item_quantity': lambda x: sorted(set(x))
    })
    .reset_index()
)

# Step 4: Flag inconsistent records
item_consistency['price_variations'] = item_consistency['item_price'].apply(len)
item_consistency['quantity_variations'] = item_consistency['item_quantity'].apply(len)

# Filter where there's more than one unique price or quantity
inconsistencies = item_consistency[
    (item_consistency['price_variations'] > 1) | 
    (item_consistency['quantity_variations'] > 1)
]

# Show top results
inconsistencies[['item_name', 'item_price', 'item_quantity']].head(10)


,item_name,item_price,item_quantity
7,PLATTER BAGELS,"[2.5, 5.0, 25.0, 75.0]","[1, 2, 10, 30]"
8,PLATTER BAKERY,"[2.0, 4.0]","[1, 2]"
9,PLATTER CHARCUTERIE & CHS,"[7.0, 14.0, 21.0, 35.0]","[1, 2, 3, 5]"
12,PLATTER SANDWICHES,"[9.0, 18.0]","[1, 2]"
14,SALAD FARMERS MRKT SM,"[25.0, 75.0]","[1, 3]"
21,SIDE CRUDITE SM,"[25.0, 108.0]","[1, 3]"
23,SIDE FRUIT SM,"[28.0, 80.0, 120.0, 400.0]","[1, 2, 3, 10]"
25,SIDE PASTA SALAD SM,"[87.0, 290.0]","[3, 10]"


In [136]:
catering_items['price_per_unit'] = catering_items['item_price'] / catering_items['item_quantity']


/var/folders/k_/5czlg1pj25d51wscvjq599040000gn/T/ipykernel_44316/2374035776.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [137]:
import pandas as pd

def flag_price_quantity_inconsistencies(df):
    """
    Flags Catering items with inconsistent pricing or quantity behavior.

    Parameters:
        df (pd.DataFrame): DataFrame with item_category, item_name, item_price, item_quantity

    Returns:
        pd.DataFrame: Grouped summary with flags
    """
    # Filter for Catering category
    catering_df = df[df["item_category"].str.lower() == "breakfast"].copy()

    # Group by item_name and collect unique price/quantity lists
    grouped = catering_df.groupby("item_name").agg({
        "item_price": lambda x: sorted(x.unique().tolist()),
        "item_quantity": lambda x: sorted(x.unique().tolist()),
    }).reset_index()

    # Add flags
    grouped["num_unique_prices"] = grouped["item_price"].apply(len)
    grouped["num_unique_quantities"] = grouped["item_quantity"].apply(len)
    grouped["price_range"] = grouped["item_price"].apply(lambda x: max(x) - min(x) if len(x) > 1 else 0)

    grouped["flag_inconsistent"] = (
        (grouped["num_unique_prices"] > 2) | 
        (grouped["num_unique_quantities"] > 2) | 
        (grouped["price_range"] > 50)
    )

    return grouped.sort_values("flag_inconsistent", ascending=False)


In [138]:
import plotly.express as px

# Call flagging function
flagged_df = flag_price_quantity_inconsistencies(order_items)

# 1. Scatter Plot: Pricing behavior per item
fig1 = px.scatter(
    order_items[order_items["item_category"].str.lower() == "breakfast"],
    x="item_quantity",
    y="item_price",
    color="item_name",
    title="📈 Catering Items – Price vs Quantity by Item Name",
    labels={"item_quantity": "Quantity", "item_price": "Price"}
)
fig1.show()

# 2. Flag Summary
flag_summary = flagged_df["flag_inconsistent"].value_counts().rename({True: "Flagged", False: "Consistent"}).reset_index()
flag_summary.columns = ["Status", "Count"]

fig2 = px.bar(
    flag_summary,
    x="Status",
    y="Count",
    color="Status",
    title="🚩 Flagged vs Consistent Catering Items"
)
fig2.show()


In [363]:
flagged_df.head(10)

,item_name,item_price,item_quantity,num_unique_prices,num_unique_quantities,price_range,flag_inconsistent
0,avocado toast,"[9.78, 11.78, 11.98, 13.78, 13.98, 14.67, 15.7...","[2, 3, 4, 5]",19,4,34.67,True
10,fresh classic,"[9.0, 13.0, 31.5, 45.5, 90.0]","[2, 3, 7, 20]",5,4,81.00,True
20,toasted bagel with cream cheese,"[7.78, 7.98, 11.67, 11.97, 15.56, 15.96, 23.34]","[2, 3, 4, 6]",7,4,15.56,True
19,southwest scramble,"[18.0, 24.0, 45.0]","[2, 5]",3,2,27.00,True
18,southwest sausage breakfast sandwich,"[9.78, 11.78, 11.98, 13.78, 13.98, 14.67, 15.7...","[2, 3, 4, 5, 6, 8, 10]",30,7,50.12,True
17,smoked ham & hot honey egg & cheese,"[9.78, 11.78, 11.98, 13.78, 13.98, 14.67, 15.7...","[2, 3, 4, 5, 6]",38,5,55.17,True
16,shakshuka,"[13.78, 15.78, 20.67, 24.0, 27.56, 34.45, 68.9]","[2, 3, 4, 5, 10]",7,5,55.12,True
15,pork roll,"[9.78, 11.78, 11.98, 13.78, 13.98, 14.67, 15.7...","[2, 3, 4, 5, 6, 7, 8, 10]",23,8,50.12,True
13,maple sausage egg & cheese,"[11.98, 13.98, 17.97, 20.97, 23.96, 29.95, 35.94]","[2, 3, 4, 5, 6]",7,5,23.96,True
1,bacon egg & cheese,"[9.78, 11.78, 11.98, 13.78, 13.98, 14.67, 15.7...","[2, 3, 4, 5, 6, 7, 8, 9, 10, 12]",61,10,68.16,True


In [88]:
def build_fact_order_items(items, options):
    """
    Build fact_order_items with corrected line prices.
    
    Logic:
    true_line_price = item_price + SUM(option_price * option_quantity)
    """
    # 1. Merge to bring in options
    merged = items.merge(options, on='lineitem_id', how='left')

    # 2. Calculate option total per row
    merged['option_total'] = merged['option_price'] * merged['option_quantity']

    # 3. Sum option total per lineitem
    option_sums = (
        merged.groupby('lineitem_id')['option_total']
        .sum()
        .reset_index()
        .rename(columns={'option_total': 'total_option_price'})
    )

    # 4. Join back with base order_items
    fact_order_items = items.merge(option_sums, on='lineitem_id', how='left')
    fact_order_items['total_option_price'] = fact_order_items['total_option_price'].fillna(0)

    # ✅ 5. Apply corrected logic: item_price is assumed to be final unless proven otherwise
    fact_order_items['true_line_price'] = (
        fact_order_items['item_price'] + fact_order_items['total_option_price']
    )

    return fact_order_items



In [ ]:
order_item_30 = order_items[order_items['item_quantity'] > 30]


In [90]:
# 
fact_order_items = build_fact_order_items(order_item_30, order_item_options)


In [97]:
fact_order_items.head(30)


,app_name,restaurant_id,creation_time_utc,order_id,user_id,printed_card_number,is_loyalty,currency,lineitem_id,item_category,item_name,item_price,item_quantity,source_table,total_option_price,true_line_price,quantity_outlier,line_price_outlier
0,Alltown Fresh,62603c5b3fbb945d37551125,2022-06-06T19:34:10.288Z,629e56b27fc21d7d5a02c2e1,None,NaN,False,USD,629e572a0d8fa14c4d429963,BBQ Sides & Extras,Side Mac N' Cheese,124.48,32,dbo.order_items,0.0,124.48,False,False
1,Alltown Fresh,62603c5b3fbb945d37551125,2022-06-06T19:34:10.288Z,629e56b27fc21d7d5a02c2e1,None,NaN,False,USD,629e5713972f532bc30346d3,BBQ Sides & Extras,Side Macaroni Salad,190.61,49,dbo.order_items,0.0,190.61,False,False
2,Alltown Fresh,62603c5b3fbb945d37551125,2022-06-06T19:34:10.288Z,629e56b27fc21d7d5a02c2e1,None,NaN,False,USD,629e571f7da483fa92026c90,BBQ Sides & Extras,Side Potato Salad,132.26,34,dbo.order_items,0.0,132.26,False,False
3,Alltown Fresh,5edff313902ad5580e9c3f30,2021-03-03T18:57:29.844Z,603fdc19505ee91d1eb8cce2,603fdbf837ab46cf2eec0150,NaN,False,USD,603fdcd5adb0d5a12db8ccdf,Bottled Beverages,COCA COLA 20OZ,142.35,65,dbo.order_items,0.0,142.35,False,False
4,Alltown Fresh,5fece84aadb0d51509c36f22,2021-10-07T19:37:58.483Z,615f4c9662e4983d105aeaa4,None,NaN,False,USD,615f4cebffd125163025817a,Bowls,Chili Chicken Bowl,3567.00,300,dbo.order_items,0.0,3567.00,False,False
5,Alltown Fresh,5e7e35f0902ad5ac017b242d,2020-08-19T12:31:35.028Z,5f3d1ba7505ee9374b7b23cc,5ee7285f505ee9f7490cf93d,NaN,False,USD,5f3d1ba84f5ee91b687b23ec,Bowls,Korean Kimchi,5000.00,500,dbo.order_items,0.0,5000.00,False,False
6,Alltown Fresh,622107b40ac81503e0369ca6,2022-05-25T11:17:11.722Z,628e1037c13c953b42750c9a,628cb6deae57d652f7093b8b,NaN,False,USD,628e10389d94cc63795e4d01,Breakfast,Breakfast Burrito*,293.37,33,dbo.order_items,3.0,296.37,False,False
7,Alltown Fresh,5e7e35f0902ad5ac017b242d,2020-06-03T12:27:22.555Z,5ed7972a4f5ee90547011c03,5ecf9eda505ee9682b445912,NaN,False,USD,5ed799d6505ee98a68cde72d,Espresso,Espresso - Double,1125.00,500,dbo.order_items,0.0,1125.00,False,False
8,Alltown Fresh,5e7e35f0902ad5ac017b242d,2020-08-04T14:53:11.864Z,5f29765737ab464e42a492c9,5eb44c8c902ad51d3c7b23d6,NaN,False,USD,5f29765837ab467240a492d6,Espresso,Espresso - Double,69.75,31,dbo.order_items,0.0,69.75,False,False
9,Alltown Fresh,5e7e35f0902ad5ac017b242d,2020-06-02T08:46:00.719Z,5ed611c8505ee9cc68872ead,5ecf9eda505ee9682b445912,NaN,False,USD,5ed611ca505ee90b68872ebb,Hot Espresso,Hot Chocolate - 12oz,245.00,50,dbo.order_items,2.9,247.90,False,False


In [92]:
def dynamic_iqr_multiplier(std, base=1.5):
    """
    Adjust IQR multiplier based on standard deviation of the group.
    """
    if std < 1:
        return base * 1.0
    elif std < 5:
        return base * 1.5
    else:
        return base * 2.0

In [93]:
def flag_outliers_dynamic_iqr(df, group_cols, target_col, flag_col_name):
    """
    Generic IQR outlier flagger with dynamic threshold based on group std deviation.
    
    Parameters:
        df: input DataFrame
        group_cols: list of columns to group by (e.g., ['item_category', 'item_name_cleaned'])
        target_col: column to evaluate (e.g., 'line_item_total', 'item_quantity')
        flag_col_name: name of the new column for outlier flag
    
    Returns:
        df_out: DataFrame with outlier flag
        thresholds_df: DataFrame with thresholds and multipliers
    """
    flagged_rows = []
    thresholds = []

    for group_keys, group in df.groupby(group_cols):
        group = group.copy()

        # Compute IQR-based dynamic threshold
        std = group[target_col].std()
        q1 = group[target_col].quantile(0.25)
        q3 = group[target_col].quantile(0.75)
        iqr = q3 - q1

        multiplier = dynamic_iqr_multiplier(std)
        lower = q1 - multiplier * iqr
        upper = q3 + multiplier * iqr

        # Flag outliers
        group[flag_col_name] = (group[target_col] < lower) | (group[target_col] > upper)

        flagged_rows.append(group)

        thresholds.append({
            **dict(zip(group_cols, group_keys)),
            'metric': target_col,
            'q1': q1,
            'q3': q3,
            'iqr': iqr,
            'std': std,
            'multiplier': multiplier,
            'lower': lower,
            'upper': upper,
            'group_size': len(group)
        })

    df_out = pd.concat(flagged_rows, ignore_index=True)
    thresholds_df = pd.DataFrame(thresholds)

    return df_out, thresholds_df


In [96]:
# Flag quantity outliers
fact_order_items, qty_thresholds = flag_outliers_dynamic_iqr(
    fact_order_items,
    group_cols=['item_category', 'item_name'],
    target_col='item_quantity',
    flag_col_name='quantity_outlier'
)

# Flag total line price outliers
fact_order_items, price_thresholds = flag_outliers_dynamic_iqr(
    fact_order_items,
    group_cols=['item_category', 'item_name'],
    target_col='true_line_price',
    flag_col_name='line_price_outlier'
)


In [68]:
fact_order_items.head(7)

,app_name,restaurant_id,order_key,user_id,order_id,lineitem_id,item_category,item_name_cleaned,size,item_price,item_quantity,date,is_loyalty,total_option_price,true_line_price,quantity_outlier,line_price_outlier
0,Alltown Fresh,5ed54933902ad53b4128658b,30660ca538438a96ceeb6bebe5ff1e73,5eac88d6902ad598127b240b,5ede8ed6505ee96b0c196d6f,5ede8edeadb0d5f31d5fdc2c,Beverages,32 oz. drink,32 oz,1.69,1.0,2020-06-08,False,0.0,1.69,False,False
1,Alltown Fresh,5edff313902ad5580e9c3f30,7ad05f0be40baa0561a1bece4620bdbd,5f1b00e5535ee93e0cb768e7,5f90822d505ee9c1282585ac,5f908236505ee9bc26258a90,Beverages,alltown fresh cold beverage 16 oz,16 oz,1.99,1.0,2020-10-21,False,0.0,1.99,False,False
2,Alltown Fresh,5edff313902ad5580e9c3f30,288590a52f53779bf41c28f3b3426a88,5f3d955c37ab46de2a7b240b,5f4e8af437ab4631306b1ba7,5f4e8b4e37ab46432c6b1c04,Beverages,alltown fresh cold beverage 16 oz,16 oz,1.99,1.0,2020-09-01,False,0.0,1.99,False,False
3,Alltown Fresh,5edff313902ad5580e9c3f30,4fec7140ea74a28836500f8436b32c60,5f1b00e5535ee93e0cb768e7,5f71f6a6adb0d5c90a7b24d8,5f71f6a637ab46dc3b7b2409,Beverages,alltown fresh cold beverage 16 oz,16 oz,1.99,1.0,2020-09-28,False,0.0,1.99,False,False
4,Alltown Fresh,5edff313902ad5580e9c3f30,bc5bc763b47731d6a68c7e7080b9ad0c,5f1b00e5535ee93e0cb768e7,5f4f8f354f5ee9892438602b,5f4f8f3e505ee91218ad9cf5,Beverages,alltown fresh cold beverage 16 oz,16 oz,1.99,1.0,2020-09-02,False,0.0,1.99,False,False
5,Alltown Fresh,5edff313902ad5580e9c3f30,36586a6457a29ce266483186784e8a0a,5f1b00e5535ee93e0cb768e7,5f611f4cadb0d55f64892296,5f611f4cadb0d59d6389228a,Beverages,alltown fresh cold beverage 16 oz,16 oz,1.99,1.0,2020-09-15,False,0.0,1.99,False,False
6,Alltown Fresh,5edff2fe4f5ee969161e8404,fda9fb838375bb04707ac686cf2ad348,5eac88d6902ad598127b240b,5f6108ab37ab46f77109fd89,5f6108af37ab46f77109fd95,Beverages,alltown fresh cold beverage 16 oz,16 oz,1.99,1.0,2020-09-15,False,0.0,1.99,False,False


In [69]:
fact_order_items[fact_order_items['line_price_outlier'] == True].head(7)

,app_name,restaurant_id,order_key,user_id,order_id,lineitem_id,item_category,item_name_cleaned,size,item_price,item_quantity,date,is_loyalty,total_option_price,true_line_price,quantity_outlier,line_price_outlier
59,Alltown Fresh,5edff313902ad5580e9c3f30,1faef7cb62fdd1b550223762dffe1dce,5ecf6d334f5ee97259ce6751,602caa8e505ee9c71587aa2c,602caa904f5ee91f65bed280,Beverages,iced tea,None,13.95,5.0,2021-02-17,False,0.0,13.95,True,True
79,Alltown Fresh,5edff313902ad5580e9c3f30,16af0c3b8d6d7ca1140f39c3e24f2133,5f3d938737ab46a5247b244a,603fcf4637ab46ab10ec0153,603fd24337ab466b15ec01a9,Beverages,lemonade,None,4.78,2.0,2021-03-03,False,0.0,4.78,True,True
95,Alltown Fresh,5edff313902ad5580e9c3f30,4ce3e07f3f5511624c1e1b0bbdb04462,5f17301437ab46423dc08543,5fa19b36505ee9dd30404787,5fa19b36505ee9dd3040478b,Beverages,lemonade,None,5.58,2.0,2020-11-03,False,0.0,5.58,True,True
100,Alltown Fresh,5edff313902ad5580e9c3f30,2dcf781d93e30cff004952f2059e1b00,5f17301437ab46423dc08543,5fa19ab537ab468725d1b803,5fa19ad037ab46e626d1b7fa,Beverages,lemonade,None,5.58,2.0,2020-11-03,False,0.0,5.58,True,True
106,Alltown Fresh,5edff313902ad5580e9c3f30,cc919c18b174bed6d4793ae5952561f3,_guest,602b7dcf4f5ee94e065db6ef,602b7dd04f5ee93e0a5db6f2,Beverages,lemonade,None,5.58,2.0,2021-02-16,False,0.0,5.58,True,True
165,Alltown Fresh,62f2ce9824813746ce6f5140,672469d007269465d9be0549875574f5,630a4ef6df28e404c4525b0f,630a4e702beab9b12504e94f,630a4e8525b1fd6a180d5563,Bowls,Wagyu Barbacoa Bowl,None,25.78,2.0,2022-08-27,False,1.0,26.78,True,True
166,Alltown Fresh,62f2ce9824813746ce6f5140,814553ab5a2adb9edb31fe860f545c94,630ce204d7d78da790002ba5,630ce29c4e6b6e41710890d3,630ce29dc3b486dc8d09b53e,Bowls,Wagyu Barbacoa Bowl,None,25.78,2.0,2022-08-29,False,1.0,26.78,True,True


In [98]:
order_items[(order_items['item_quantity'] > 7) & (order_items['item_quantity'] <= 10)].count()


app_name               151
restaurant_id          151
creation_time_utc      151
order_id               151
user_id                130
printed_card_number     10
is_loyalty             151
currency               151
lineitem_id            151
item_category          151
item_name              151
item_price             151
item_quantity          151
source_table           151
dtype: int64

In [195]:
order_items[order_items['item_price'] < 1].count()

app_name               174
restaurant_id          174
creation_time_utc      174
order_id               174
user_id                174
printed_card_number    174
is_loyalty             174
currency               174
lineitem_id            174
item_category          174
item_name              174
item_price             174
item_quantity          174
source_table           174
date                   174
date_key               174
item_name_cleaned      174
size                    13
order_key              174
dtype: int64

In [202]:
order_items[order_items['item_price'] < 1].count()


app_name               174
restaurant_id          174
creation_time_utc      174
order_id               174
user_id                174
printed_card_number    174
is_loyalty             174
currency               174
lineitem_id            174
item_category          174
item_name              174
item_price             174
item_quantity          174
source_table           174
date                   174
date_key               174
item_name_cleaned      174
size                    13
order_key              174
dtype: int64

In [ ]:
order_items[(order_items['item_price'] > 0)&(order_items['item_price'] < 1)].count()


app_name               18
restaurant_id          18
creation_time_utc      18
order_id               18
user_id                18
printed_card_number    18
is_loyalty             18
currency               18
lineitem_id            18
item_category          18
item_name              18
item_price             18
item_quantity          18
source_table           18
date                   18
date_key               18
item_name_cleaned      18
size                    6
order_key              18
dtype: int64

In [201]:

order_items[(order_items['item_price'] > 0)&(order_items['item_price'] < 1)].count()

app_name               18
restaurant_id          18
creation_time_utc      18
order_id               18
user_id                18
printed_card_number    18
is_loyalty             18
currency               18
lineitem_id            18
item_category          18
item_name              18
item_price             18
item_quantity          18
source_table           18
date                   18
date_key               18
item_name_cleaned      18
size                    6
order_key              18
dtype: int64

In [205]:
import pandas as pd

def remove_low_price_items(df: pd.DataFrame, price_threshold: float = 1.0) -> pd.DataFrame:
    """
    Remove rows where item_price is less than a given threshold.

    Args:
        df (pd.DataFrame): Input Pandas DataFrame.
        price_threshold (float): Minimum acceptable item_price. Default is 1.0.

    Returns:
        pd.DataFrame: Filtered DataFrame with item_price >= threshold.
    """
    return df[df["item_price"] >= price_threshold]
order_items = remove_low_price_items(order_items, price_threshold=1.0)


In [206]:
import pandas as pd

def remove_invalid_price_rows(
    df: pd.DataFrame,
    min_price: float = 1.0,
    max_price: float = 100.0
) -> pd.DataFrame:
    """
    Removes rows with invalid item_price values in a Pandas DataFrame:
    - Null or empty string
    - Less than min_price
    - Greater than max_price

    Args:
        df (pd.DataFrame): Input Pandas DataFrame.
        min_price (float): Minimum allowed item_price.
        max_price (float): Maximum allowed item_price.

    Returns:
        pd.DataFrame: Cleaned DataFrame.
    """
    df = df.copy()
    df["item_price"] = pd.to_numeric(df["item_price"], errors="coerce")  # convert to float, force invalid to NaN
    return df[
        (df["item_price"].notnull()) &
        (df["item_price"] >= min_price) &
        (df["item_price"] <= max_price)
    ]
order_items = remove_invalid_price_rows(order_items, min_price=1.0, max_price=100.0)

In [364]:
import pandas as pd
import numpy as np
import yaml

# Define the function before using it
def convert_thresholds_to_plain_types(thresholds):
    def to_plain(val):
        if pd.isna(val):
            return None
        return float(val) if isinstance(val, (np.float64, np.float32)) else int(val)

    plain_thresholds = {}
    for category, items in thresholds.items():
        plain_thresholds[category] = {}
        for item, metrics in items.items():
            plain_thresholds[category][item] = {
                "price": {k: to_plain(v) for k, v in metrics["price"].items()},
                "quantity": {k: to_plain(v) for k, v in metrics["quantity"].items()}
            }
    return plain_thresholds
# Lower case the item_category and item_name columns
order_items['item_category'] = order_items['item_category'].str.lower()
order_items['item_name'] = order_items['item_name'].str.lower()
# Step 1: Group by category and item_name
grouped = order_items.groupby(["item_category", "item_name"])

# Step 2: Calculate thresholds
thresholds = {}
for (category, item), group in grouped:
    if category not in thresholds:
        thresholds[category] = {}

    thresholds[category][item] = {
        "price": {
            "min": group["item_price"].min(),
            "max": group["item_price"].max(),
            "mean": group["item_price"].mean(),
            "stddev": group["item_price"].std()
        },
        "quantity": {
            "min": group["item_quantity"].min(),
            "max": group["item_quantity"].max(),
            "mean": group["item_quantity"].mean(),
            "stddev": group["item_quantity"].std()
        }
    }

# Now safely convert and dump
plain_thresholds = convert_thresholds_to_plain_types(thresholds)
with open('thresholds.yaml', 'w') as file:
    yaml.dump(plain_thresholds, file, sort_keys=False, default_flow_style=False)
yaml_string = yaml.dump(plain_thresholds, sort_keys=False, default_flow_style=False)
print(yaml_string[:1000])


bbq plates:
  pulled pork plate:
    price:
      min: 19
      max: 29
      mean: 21.938000000000002
      stddev: 4
    quantity:
      min: 2
      max: 3
      mean: 2.2
      stddev: 0
  smoked brisket plate:
    price:
      min: 29
      max: 29
      mean: 29.98
      stddev: 0
    quantity:
      min: 2
      max: 2
      mean: 2.0
      stddev: 0
bbq sides & extras:
  side b & b pickles:
    price:
      min: 3
      max: 3
      mean: 3.9133333333333336
      stddev: 0
    quantity:
      min: 2
      max: 2
      mean: 2.0
      stddev: 0
  side banana pudding:
    price:
      min: 9
      max: 19
      mean: 12.090000000000003
      stddev: 3
    quantity:
      min: 2
      max: 4
      mean: 2.4285714285714284
      stddev: 0
  side coleslaw:
    price:
      min: 7
      max: 11
      mean: 8.9275
      stddev: 2
    quantity:
      min: 2
      max: 3
      mean: 2.25
      stddev: 0
  side corn bread:
    price:
      min: 3
      max: 11
      mean: 4.4230769230769

In [209]:
def create_unique_category_item_pairs_to_csv(df, output_path):
    """
    Extract unique (item_category, item_name) pairs and write them to a CSV file.

    Parameters:
        df (pd.DataFrame): Input DataFrame.
        output_path (str): Destination file path (e.g., "category_item_pairs.csv").
    """
    # Drop missing values
    cleaned_df = df[['item_category', 'item_name']].dropna()

    # Normalize values
    cleaned_df['item_category'] = cleaned_df['item_category'].str.lower().str.strip()
    cleaned_df['item_name'] = cleaned_df['item_name'].str.lower().str.strip()

    # Drop duplicates
    unique_df = cleaned_df.drop_duplicates()

    # Save to CSV
    unique_df.to_csv(output_path, index=False)
    print(f"✅ Saved {len(unique_df)} unique category-item pairs to '{output_path}'")

# Example usage
create_unique_category_item_pairs_to_csv(order_items, "category_item_pairs.csv")


✅ Saved 324 unique category-item pairs to 'category_item_pairs.csv'


In [21]:
import pandas as pd
import numpy as np

def generate_price_summary_pandas(df):
    summary_rows = []

    grouped = df.groupby(['item_category', 'item_name_cleaned'])

    for (category, name), group in grouped:
        if group.empty:
            continue

        item_prices = group['item_price'].dropna()
        item_quantities = group['item_quantity'].dropna()

        if len(item_prices) < 3:
            continue  # not enough data for quantile estimation

        q25, q50, q75 = np.quantile(item_prices, [0.25, 0.5, 0.75])

        item_price_mean = item_prices.mean()
        item_price_std = item_prices.std()
        item_price_min = item_prices.min()
        item_price_max = item_prices.max()

        item_quantity_mean = item_quantities.mean()
        item_quantity_std = item_quantities.std()
        item_quantity_min = item_quantities.min()
        item_quantity_max = item_quantities.max()

        # Heuristic for guessed price
        guessed_price = round(q50, 2) if (
            item_price_min < 0.8 * q25 or item_price_max > 1.5 * q75
        ) else round(item_price_mean, 2)

        summary_rows.append((
            category, name,
            item_price_mean, item_price_std, item_price_min, item_price_max,
            q25, q50, q75,
            item_quantity_mean, item_quantity_std, item_quantity_min, item_quantity_max,
            guessed_price
        ))

    return pd.DataFrame(summary_rows, columns=[
        "item_category", "item_name", "item_price_mean", "item_price_std", "item_price_min", "item_price_max",
        "q25", "q50", "q75", "item_quantity_mean", "item_quantity_std", "item_quantity_min",
        "item_quantity_max", "guessed_price"
    ])


In [22]:

order_items_price_summary = generate_price_summary_pandas(order_items_cat) 
order_items_price_summary.head(7)


,item_category,item_name,item_price_mean,item_price_std,item_price_min,item_price_max,q25,q50,q75,item_quantity_mean,item_quantity_std,item_quantity_min,item_quantity_max,guessed_price
0,Beverages,alltown fresh cold beverage 16 oz,1.99000,6.895140e-16,1.99,1.99,1.99,1.99,1.99,1.000000,0.000000,1.0,1.0,1.99
1,Beverages,alltown fresh cold beverage 24 oz,2.39000,0.000000e+00,2.39,2.39,2.39,2.39,2.39,1.000000,0.000000,1.0,1.0,2.39
2,Beverages,alltown fresh cold beverage 32 oz,2.79000,4.586534e-16,2.79,2.79,2.79,2.79,2.79,1.000000,0.000000,1.0,1.0,2.79
3,Beverages,iced tea,3.94350,3.119044e+00,1.99,13.95,1.99,2.59,4.18,1.650000,1.182103,1.0,5.0,2.59
4,Beverages,lemonade,2.60587,1.018071e+00,1.99,5.58,1.99,1.99,2.79,1.152174,0.363158,1.0,2.0,1.99
5,Bowls,Chicken Teriyaki Bowl,11.89000,5.483568e-15,11.89,11.89,11.89,11.89,11.89,1.000000,0.000000,1.0,1.0,11.89
6,Bowls,Hot Honey Quinoa Bowl,11.89000,0.000000e+00,11.89,11.89,11.89,11.89,11.89,1.000000,0.000000,1.0,1.0,11.89


In [173]:
def calculate_total_prices(df):
    df = df.copy()
    df['total_line_price'] = df['item_price'] + (df['option_price'].fillna(0.00) * df['option_quantity'].fillna(0))
    df['total_price'] = (df['item_price'] * df['item_quantity']) + (df['option_price'].fillna(0.0) * df['option_quantity'].fillna(0))
    return df


In [175]:
# total_prices_summary = calculate_total_prices(merged_stat).head(7)

In [ ]:
# call funtion generate_price_summary_pandas
# order_items_price_summary = generate_price_summary_pandas(order_items_cat)

In [176]:
# order_items_price_summary.head(10)

In [28]:
order_items_price_summary[order_items_price_summary['item_category'] == 'catering']

,item_category,item_name,item_price_mean,item_price_std,item_price_min,item_price_max,q25,q50,q75,item_quantity_mean,item_quantity_std,item_quantity_min,item_quantity_max,guessed_price
144,catering,hot pumpkin latte 16 oz,3.000000,0.000000,3.0,3.0,3.0,3.0,3.0,1.000000,0.000000,1.0,1.0,3.0
145,catering,platter bagels,2.750000,0.790569,2.5,5.0,2.5,2.5,2.5,1.100000,0.316228,1.0,2.0,2.5
146,catering,platter bakery,2.285714,0.755929,2.0,4.0,2.0,2.0,2.0,1.142857,0.377964,1.0,2.0,2.0
147,catering,platter charcuterie & chs,12.600000,8.024961,7.0,35.0,7.0,7.0,14.0,1.800000,1.146423,1.0,5.0,7.0
148,catering,platter fruit,3.000000,0.000000,3.0,3.0,3.0,3.0,3.0,1.000000,0.000000,1.0,1.0,3.0
149,catering,platter salmon,6.000000,0.000000,6.0,6.0,6.0,6.0,6.0,1.000000,0.000000,1.0,1.0,6.0
150,catering,platter sandwiches,10.285714,3.401680,9.0,18.0,9.0,9.0,9.0,1.142857,0.377964,1.0,2.0,9.0
151,catering,salad killer cobb sm,30.000000,0.000000,30.0,30.0,30.0,30.0,30.0,1.000000,0.000000,1.0,1.0,30.0


In [29]:
order_items_price_summary[order_items_price_summary['item_price_mean'] > 10].head(7)

,item_category,item_name,item_price_mean,item_price_std,item_price_min,item_price_max,q25,q50,q75,item_quantity_mean,item_quantity_std,item_quantity_min,item_quantity_max,guessed_price
5,Bowls,Chicken Teriyaki Bowl,11.890000,5.483568e-15,11.89,11.89,11.89,11.89,11.89,1.000000,0.000000,1.0,1.0,11.89
6,Bowls,Hot Honey Quinoa Bowl,11.890000,0.000000e+00,11.89,11.89,11.89,11.89,11.89,1.000000,0.000000,1.0,1.0,11.89
8,Bowls,Wagyu Barbacoa Bowl,13.510000,4.339743e+00,11.89,25.78,11.89,11.89,12.39,1.105263,0.315302,1.0,2.0,11.89
27,Kids,kids pasta,12.000000,6.196773e+00,4.00,16.00,7.00,16.00,16.00,3.000000,1.549193,1.0,4.0,16.00
36,Kombuchas,bluebernie kombucha,10.196667,6.650000e+00,3.99,19.95,3.99,7.98,15.96,2.555556,1.666667,1.0,5.0,10.20
45,Salads,chop salad,10.556667,1.154701e+00,9.89,11.89,9.89,9.89,10.89,1.000000,0.000000,1.0,1.0,10.56
46,Salads,kale caesar salad,10.890000,1.154701e+00,9.89,11.89,9.89,10.89,11.89,1.000000,0.000000,1.0,1.0,10.89


## Next, let's move on to the categorical features


#    

In [ ]:
# Select only numerical columns (excluding 'printed_card_number')
# numerical_columns = order_items_allfresh.select_dtypes(include=['float64', 'int64']).columns
# numerical_columns = order_items_allfresh.drop(['printed_card_number'], errors='ignore')  # optional cleanup

# # Filter top categories only once
# top_10_categories = order_items_allfresh['item_category'].value_counts().nlargest(10).index
# filtered_cat = order_items_allfresh[order_items_allfresh['item_category'].isin(top_10_categories)]
numerical_columns = order_items_price_summary.select_dtypes(include=['float64', 'int64']).columns
top_10_categories = order_items_cat['item_category'].value_counts().nlargest(10).index
order_items_price_summary_box = order_items_price_summary[order_items_price_summary['item_category'].isin(top_10_categories)]
# Plot boxplots for each numerical column grouped by item_category
for column in numerical_columns:
    fig = px.box(
        data_frame=order_items_price_summary_box,
        y=column,
        x='item_category',
        color='item_category',
        facet_col='item_category',
        facet_col_wrap=2,
        height=900,
        width=1200,
        title=f'Distribution of {column} by Item Category (Top 5)',
        log_y=True if column == 'item_price' else False  # optional: log scale for skewed price
    )

    fig.update_layout(
        xaxis_title='Item Category',
        yaxis_title=column
    )

    fig.show()


In [ ]:
# select bowls and  grouped by item_name_cleaned
bowls_summary = order_items_price_summary[order_items_price_summary['item_category'] == 'bowls']
numerical_columns = bowls_summary.select_dtypes(include=['float64', 'int64']).columns
top_10_categories = bowls_summary['item_category'].value_counts().nlargest(10).index
order_items_price_summary_box = bowls_summary[bowls_summary['item_category'].isin(top_10_categories)]
# Plot boxplots for each numerical column grouped by item_category
for column in numerical_columns:
    fig = px.box(
        data_frame=order_items_price_summary_box,
        y=column,
        x='item_category',
        color='item_category',
        facet_col='item_category',
        facet_col_wrap=2,
        height=900,
        width=1200,
        title=f'Distribution of {column} by Item Category (Top 5)',
        log_y=True if column == 'item_price' else False  # optional: log scale for skewed price
    )

    fig.update_layout(
        xaxis_title='Item Category',
        yaxis_title=column
    )

    fig.show()

In [ ]:
order_items_coldbrew[order_items_coldbrew['item_price'] > 1].head(10)


In [ ]:
# list the cold brew categories
cold_brew_categories = order_items_coldbrew['item_name'].unique()
cold_brew_categories

In [ ]:
# Summary statistics for numerical columns by item_category (Top 5 only)
numerical_summary = order_items_allfresh.groupby('item_category')[numerical_columns].agg(
    ['mean', 'std', 'min', 'max', 
     lambda x: x.quantile(0.25),
     lambda x: x.quantile(0.5),
     lambda x: x.quantile(0.75)]
)

# Rename columns for clarity
numerical_summary.columns = ['_'.join(col).strip() for col in numerical_summary.columns.values]
numerical_summary = numerical_summary.reset_index()

#mmary by Item Category")
numerical_summary


# **Category Features Analysis Summary**
## Key Observations
1. Reasonable Categories
    - Many categories like `BBQ Plates`, `Bowls`, `Breakfast`, `Sandwiches`, `Smoothies`, etc., show a reasonable price range and standard deviation, suggesting consistent pricing behavior.
## Concerning Anomalies & Errors
### A. Extreme Price Outliers
2. Several categories have abnormally high maximum prices, far exceeding the mean — likely data entry errors or special/bulk pricing.

Examples:


In [ ]:
# where
# Summary statistics for numerical columns by item_category (Top 5 only)
numerical_summary = order_items_allfresh.groupby('item_category')[numerical_columns].agg(
    ['mean', 'std',  'max', 
     lambda x: x.quantile(0.25),
     lambda x: x.quantile(0.5),
     lambda x: x.quantile(0.75)]
)

# Rename columns for clarity
numerical_summary.columns = ['_'.join(col).strip() for col in numerical_summary.columns.values]
numerical_summary = numerical_summary.reset_index()

#mmary by Item Category")
numerical_summary[numerical_summary['item_price_std'] >= 24]


**Concern:** 
- These values are implausible for typical food items.

 **Action:** 
 - Filter for prices over $100 for review.

In [ ]:
# Identify extreme outlier prices
# extreme_prices = order_items_allfresh[order_items_allfresh['item_price'] > 100]
# extreme_prices

#### Invalid or Noisy Category Labels
- Some item_category values contain URLs or typos, suggesting poor data hygien

**Concern:** 
- These should be cleaned or consolidated.

**Action:**

In [ ]:
# Detect category labels with URLs or strange characters
# Detect unusual category labels with URLs, special characters, or abnormal length
def is_weird_category(cat):
    if not isinstance(cat, str):
        return False
    return (
        'http' in cat.lower() or
        '`' in cat or
        len(cat.strip()) > 30
    )

# Apply the filter
weird_labels = [cat for cat in order_items_allfresh['item_category'].unique() if is_weird_category(cat)]

# Display the results
print("Suspicious or malformed item_category labels:")
for label in weird_labels:
    print(f"- {label}")
    


#### Zero or Implausible Prices
- Some items are priced at `$0.00`, which may represent missing or incorrectly recorded prices.

In [ ]:
# Find items with $0 price
# zero_price = order_items_allfresh[order_items_allfresh['item_price'] == 0]
# zero_price

**Concern:** These entries could distort averages and need review.

#### Quantity Outliers
- Unusually large quantities are present in some categories `(e.g. 300, 500 units)`, which may be valid for bulk orders but need validation.

In [ ]:
# Identify quantity outliers
# high_quantity = order_items_allfresh[order_items_allfresh['item_quantity'] > 50]
# high_quantity

#### E. NaNs in Std Dev
- Some categories `(e.g., Hot Coffee, Iced Coffee)` show NaN in std deviation, suggesting only 1 record was present. This limits statistical interpretation.

### Final Recommendations
- Clean inconsistent category labels and group related categories (e.g., merge "Sqalads" with "Salads").

- Investigate extreme prices and quantities above thresholds like $100 and QTY > 50.

- Remove or impute missing/zero prices depending on business rules.

- Drop or fix singleton records when statistical summaries are not meaningful.

# ***Transform Workshop***

In [ ]:
import re

def clean_category_label(label):
    if not isinstance(label, str):
        return label
    # Remove URLs
    label = re.sub(r'http\S+', '', label)
    # Remove backticks and extra whitespace
    label = label.replace('`', '').strip()
    # Optionally truncate very long labels
    return label[:30]  # Keep first 30 characters if still too long

# Apply cleaning function to item_category
order_items_allfresh['item_category'] = order_items_allfresh['item_category'].apply(clean_category_label)

# Optional: View top unique cleaned categories
print("Unique cleaned categories:")
print(order_items_allfresh['item_category'].value_counts().head(30))


In [ ]:
# filter category Smoothies and all distinct item names
smoothies_category = order_items_allfresh[order_items_allfresh['item_category'] == 'subs']
distinct_item_names = smoothies_category['item_name'].unique()
print("Distinct item names in Salads category Create a Mapping:")
for name in distinct_item_names:
    print(f"- {name}") 
# order_items_allfresh.head()
# order_items_allfresh = order_items_allfresh.drop('app_name', axis=1)  


In [ ]:
# group by item_category and item_name for Soda, Juices & Kombucha categories
soda_juices_kombucha = order_items_allfresh[
    order_items_allfresh['item_category'].str.contains('Milk', case=False, na=False)
]
# Group by item_category and item_name for Soda, Juices & Kombucha categories
soda_juices_kombucha_summary = soda_juices_kombucha.groupby(
    ['item_category', 'item_name']
)[numerical_columns].agg(
    ['mean', 'std', 'min', 'max',
     lambda x: x.quantile(0.25),
     lambda x: x.quantile(0.5),
     lambda x: x.quantile(0.75)]
).reset_index() 
# Rename columns for clarity
soda_juices_kombucha_summary.columns = ['_'.join(col).strip() for col in soda_juices_kombucha_summary.columns.values]
# Sort by mean item_price
soda_juices_kombucha_summary = soda_juices_kombucha_summary.sort_values(
    by='item_price_mean', ascending=False
)   
# Display top 20 items in Soda, Juices & Kombucha categories
soda_juices_kombucha_summary.head(20)   


In [ ]:
import pandas as pd

def generate_price_summary(df, item_col='item_name', cat_col='item_category'):
    """
    Generate price + quantity summary with quantiles and guessed price per item.

    Args:
        df (pd.DataFrame): Raw cleaned order item data.
        item_col (str): Column name for item_name.
        cat_col (str): Column name for item_category.

    Returns:
        pd.DataFrame: Summary with statistics and guessed price column.
    """

    # Define numeric columns
    numerical_columns = ["item_price", "item_quantity"]

    # Compute summary with aggregation
    summary = df.groupby([cat_col, item_col])[numerical_columns].agg(
        ['mean', 'std', 'min', 'max', 
         lambda x: x.quantile(0.25),
         lambda x: x.quantile(0.5),
         lambda x: x.quantile(0.75)]
    )

    # Rename columns
    summary.columns = ['_'.join([col[0], col[1] if isinstance(col[1], str) else f"<lambda_{i}>"]) 
                       for i, col in enumerate(summary.columns)]
    summary = summary.reset_index()

    # Define guessed price logic
    def guess_price(row):
        try:
            q25 = row['item_price_<lambda_0>']
            q50 = row['item_price_<lambda_1>']
            q75 = row['item_price_<lambda_2>']
            min_price = row['item_price_min']
            max_price = row['item_price_max']
            mean_price = row['item_price_mean']

            if min_price < 0.8 * q25 or max_price > 1.5 * q75:
                return round(q50, 2)
            else:
                return round(mean_price, 2)
        except:
            return None

    # Apply guessed price
    summary['guessed_price'] = summary.apply(guess_price, axis=1)

    return summary


In [ ]:
bbq_summary = generate_price_summary(order_items_allfresh)

# Filter for BBQ Plates
bbq_summary[bbq_summary['item_category'].str.contains('Bowls')] \
    .sort_values(by='item_price_mean', ascending=False) \
    .head(20)


In [ ]:
# bbq_summary.sort_values(by='guessed_price', ascending=False).head(10)


In [ ]:
import pandas as pd
import numpy as np

def flag_anomalies(summary_df):
    def detect_flags(row):
        flags = []
        
        # Rule 1: Zero price but meaningful mean
        if row['item_price_min'] == 0 and row['item_price_mean'] > 1:
            flags.append('R1')
        
        # Rule 2: Max price way above average (outlier)
        if row['item_price_max'] > row['item_price_mean'] + 3 * row['item_price_std']:
            flags.append('R2')

        # Rule 3: Std dev of price higher than mean (too volatile)
        if row['item_price_std'] > row['item_price_mean']:
            flags.append('R3')
        
        # Rule 4: Huge quantity spikes
        if row['item_quantity_max'] > 50 and row['item_quantity_mean'] < 5:
            flags.append('R4')

        # Rule 5: Prices absurdly high (excluding Catering/Tobacco)
        if row['item_price_max'] > 500 and row['item_category'] not in ['Catering', 'Tobacco']:
            flags.append('R5')

        # Rule 6: Quantity variance too high
        if row['item_quantity_std'] > row['item_quantity_mean']:
            flags.append('R6')

        # Rule 7: No variation at all (possible test data)
        if row['item_price_min'] == row['item_price_max'] and row['item_price_std'] == 0:
            flags.append('R7')

        # Rule 8: NaN std deviation
        if pd.isna(row['item_price_std']) or pd.isna(row['item_quantity_std']):
            flags.append('R8')
        
        return flags

    summary_df['flags'] = summary_df.apply(detect_flags, axis=1)
    flagged_rows = summary_df[summary_df['flags'].apply(lambda x: len(x) > 0)]

    return flagged_rows


In [ ]:
# Assume your existing DataFrame is named `numerical_summary`
flagged_df = flag_anomalies(numerical_summary)

# Show top 10 problematic entries
print(flagged_df[['item_category', 'item_price_mean', 'item_price_min', 'item_price_max', 'flags']].head(10))

# Optional: Export to CSV
flagged_df.to_csv("flagged_outliers.csv", index=False)


In [ ]:
flagged_df.head(10)

## Bowl Category Standardization

In [ ]:
# Flagging rules
df_flagged = order_items_allfresh[
    (order_items_allfresh['item_price'] > 100) |
    (order_items_allfresh['item_quantity'] > 50) |
    (order_items_allfresh['item_price'] == 0)
]
df_flagged = df_flagged[['restaurant_id', 'order_id', 'item_name', 'item_category', 'item_price', 'item_quantity']]
df_flagged

In [177]:
import pandas as pd
import numpy as np

def dynamic_iqr_multiplier(std, base=1.5):
    """
    Adjust IQR multiplier based on standard deviation of the group.
    """
    if std < 1:
        return base * 1.0
    elif std < 5:
        return base * 1.5
    else:
        return base * 2.0

def flag_outliers_dynamic_iqr(df, group_cols, price_col='item_price', qty_col='item_quantity'):
    """
    Flags outliers in price and quantity using dynamic IQR threshold per group.
    
    Parameters:
        df: DataFrame with item data
        group_cols: list of columns to group by (e.g., ['item_category', 'item_name_cleaned'])
        price_col: column for item price
        qty_col: column for item quantity
    
    Returns:
        df_out: Original DataFrame with `price_outlier` and `quantity_outlier` columns
        thresholds_df: DataFrame of thresholds per group
    """
    flagged_rows = []
    thresholds = []

    for group_keys, group in df.groupby(group_cols):
        group = group.copy()
        
        # --- Price Outlier Logic ---
        price_std = group[price_col].std()
        price_q1 = group[price_col].quantile(0.25)
        price_q3 = group[price_col].quantile(0.75)
        price_iqr = price_q3 - price_q1
        price_multiplier = dynamic_iqr_multiplier(price_std)
        price_lower = price_q1 - price_multiplier * price_iqr
        price_upper = price_q3 + price_multiplier * price_iqr
        
        # --- Quantity Outlier Logic ---
        qty_std = group[qty_col].std()
        qty_q1 = group[qty_col].quantile(0.25)
        qty_q3 = group[qty_col].quantile(0.75)
        qty_iqr = qty_q3 - qty_q1
        qty_multiplier = dynamic_iqr_multiplier(qty_std)
        qty_lower = qty_q1 - qty_multiplier * qty_iqr
        qty_upper = qty_q3 + qty_multiplier * qty_iqr

        # Flag rows
        group['price_outlier'] = (group[price_col] < price_lower) | (group[price_col] > price_upper)
        group['quantity_outlier'] = (group[qty_col] < qty_lower) | (group[qty_col] > qty_upper)

        # Append to results
        flagged_rows.append(group)

        # Log thresholds
        thresholds.append({
            **dict(zip(group_cols, group_keys)),
            'price_q1': price_q1,
            'price_q3': price_q3,
            'price_iqr': price_iqr,
            'price_multiplier': price_multiplier,
            'price_lower': price_lower,
            'price_upper': price_upper,
            'qty_q1': qty_q1,
            'qty_q3': qty_q3,
            'qty_iqr': qty_iqr,
            'qty_multiplier': qty_multiplier,
            'qty_lower': qty_lower,
            'qty_upper': qty_upper,
            'group_size': len(group)
        })

    df_out = pd.concat(flagged_rows, ignore_index=True)
    thresholds_df = pd.DataFrame(thresholds)

    return df_out, thresholds_df


In [179]:
# ✅ 1. Join order_items + order_item_options
# ✅ 2. Calculate true line_item_total (including options)
# ✅ 3. Update flag_outliers_dynamic_iqr() to use this new total
# ✅ 4. Produce a final cleaned fact_order_items table
# 🧱 Step 1: Build fact_order_items With True Line Price

def build_fact_order_items(order_items, order_item_options):
    """
    Build fact_order_items by computing true line totals including options.
    
    Parameters:
        order_items: DataFrame with item base prices and quantities
        order_item_options: DataFrame with modifiers (e.g., size, toppings)
    
    Returns:
        fact_order_items: DataFrame with enriched fields
    """
    # 1. Merge to attach option info
    merged = order_items.merge(order_item_options, on='lineitem_id', how='left')

    # 2. Compute option total per row (per modifier)
    merged['option_total'] = merged['option_price'] * merged['option_quantity']

    # 3. Sum option_total per lineitem_id
    option_sums = (
        merged.groupby('lineitem_id')['option_total']
        .sum()
        .reset_index()
        .rename(columns={'option_total': 'total_option_price'})
    )

    # 4. Merge back with order_items
    fact_order_items = order_items.merge(option_sums, on='lineitem_id', how='left')
    fact_order_items['total_option_price'] = fact_order_items['total_option_price'].fillna(0)

    # 5. Compute final total line price
    fact_order_items['line_item_total'] = (
        fact_order_items['item_price'] * fact_order_items['item_quantity']
        + fact_order_items['total_option_price']
    )

    return fact_order_items

# fact_order_items = build_fact_order_items(order_items, order_item_options)
# fact_order_items[['lineitem_id', 'item_price', 'item_quantity', 'total_option_price', 'line_item_total']].head()

# 🛠️ Step 2: Modify flag_outliers_dynamic_iqr to Use line_item_total
# We’ll allow any target column (price, quantity, or total).


def flag_outliers_dynamic_iqr(df, group_cols, target_col, flag_col_name):
    """
    Generic IQR outlier flagger with dynamic threshold based on group std deviation.
    
    Parameters:
        df: input DataFrame
        group_cols: list of columns to group by (e.g., ['item_category', 'item_name_cleaned'])
        target_col: column to evaluate (e.g., 'line_item_total', 'item_quantity')
        flag_col_name: name of the new column for outlier flag
    
    Returns:
        df_out: DataFrame with outlier flag
        thresholds_df: DataFrame with thresholds and multipliers
    """
    flagged_rows = []
    thresholds = []

    for group_keys, group in df.groupby(group_cols):
        group = group.copy()

        # Compute IQR-based dynamic threshold
        std = group[target_col].std()
        q1 = group[target_col].quantile(0.25)
        q3 = group[target_col].quantile(0.75)
        iqr = q3 - q1

        multiplier = dynamic_iqr_multiplier(std)
        lower = q1 - multiplier * iqr
        upper = q3 + multiplier * iqr

        # Flag outliers
        group[flag_col_name] = (group[target_col] < lower) | (group[target_col] > upper)

        flagged_rows.append(group)

        thresholds.append({
            **dict(zip(group_cols, group_keys)),
            'metric': target_col,
            'q1': q1,
            'q3': q3,
            'iqr': iqr,
            'std': std,
            'multiplier': multiplier,
            'lower': lower,
            'upper': upper,
            'group_size': len(group)
        })

    df_out = pd.concat(flagged_rows, ignore_index=True)
    thresholds_df = pd.DataFrame(thresholds)

  


In [104]:
# Flag quantity outliers
fact_order_items, qty_thresholds = flag_outliers_dynamic_iqr(
    fact_order_items,
    group_cols=['item_category', 'item_name'],
    target_col='item_quantity',
    flag_col_name='quantity_outlier'
)

# Flag total line price outliers
fact_order_items, price_thresholds = flag_outliers_dynamic_iqr(
    fact_order_items,
    group_cols=['item_category', 'item_name'],
    target_col='line_item_total',
    flag_col_name='line_price_outlier'
)


NameError: name 'flag_outliers_dynamic_iqr' is not defined